**Import Required Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

**Read the Dataset**


In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='jxU6sXfYrzwi-9LAkex2pOgcADk064zAqCqUfZV_tOW0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'carresale-donotdelete-pr-pfe1qqzhfz7ssa'
object_key = 'autos.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,24-03-2016 11:52,Golf_3_1.6,privat,Angebot,480.0,test,NaN,1993.0,manuell,0.0,golf,150000.0,0.0,benzin,volkswagen,NaN,24-03-2016 00:00,0.0,70435.0,07-04-2016 03:16
1,24-03-2016 10:58,A5_Sportback_2.7_Tdi,privat,Angebot,18300.0,test,coupe,2011.0,manuell,190.0,NaN,125000.0,5.0,diesel,audi,ja,24-03-2016 00:00,0.0,66954.0,07-04-2016 01:46
2,14-03-2016 12:52,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800.0,test,suv,2004.0,automatik,163.0,grand,125000.0,8.0,diesel,jeep,NaN,14-03-2016 00:00,0.0,90480.0,05-04-2016 12:47
3,17-03-2016 16:54,GOLF_4_1_4__3TÜRER,privat,Angebot,1500.0,test,kleinwagen,2001.0,manuell,75.0,golf,150000.0,6.0,benzin,volkswagen,nein,17-03-2016 00:00,0.0,91074.0,17-03-2016 17:40
4,31-03-2016 17:25,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600.0,test,kleinwagen,2008.0,manuell,69.0,fabia,90000.0,7.0,diesel,skoda,nein,31-03-2016 00:00,0.0,60437.0,06-04-2016 10:17


**Cleaning the Dataset**

In [3]:
print(df.seller.value_counts())

privat        101235
gewerblich         1
Name: seller, dtype: int64


In [4]:
df [ df.seller != 'gewerblich']

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,24-03-2016 11:52,Golf_3_1.6,privat,Angebot,480.0,test,NaN,1993.0,manuell,0.0,golf,150000.0,0.0,benzin,volkswagen,NaN,24-03-2016 00:00,0.0,70435.0,07-04-2016 03:16
1,24-03-2016 10:58,A5_Sportback_2.7_Tdi,privat,Angebot,18300.0,test,coupe,2011.0,manuell,190.0,NaN,125000.0,5.0,diesel,audi,ja,24-03-2016 00:00,0.0,66954.0,07-04-2016 01:46
2,14-03-2016 12:52,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800.0,test,suv,2004.0,automatik,163.0,grand,125000.0,8.0,diesel,jeep,NaN,14-03-2016 00:00,0.0,90480.0,05-04-2016 12:47
3,17-03-2016 16:54,GOLF_4_1_4__3TÜRER,privat,Angebot,1500.0,test,kleinwagen,2001.0,manuell,75.0,golf,150000.0,6.0,benzin,volkswagen,nein,17-03-2016 00:00,0.0,91074.0,17-03-2016 17:40
4,31-03-2016 17:25,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600.0,test,kleinwagen,2008.0,manuell,69.0,fabia,90000.0,7.0,diesel,skoda,nein,31-03-2016 00:00,0.0,60437.0,06-04-2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101231,20-03-2016 17:36,Opel_Mokka_1.4_Turbo_ecoFLEX_Start/Stop_Color_...,privat,Angebot,20500.0,test,suv,2015.0,manuell,140.0,andere,10000.0,10.0,benzin,opel,nein,20-03-2016 00:00,0.0,65428.0,05-04-2016 19:47
101232,29-03-2016 14:42,BMW_325i,privat,Angebot,4000.0,test,cabrio,1996.0,manuell,192.0,3er,150000.0,3.0,benzin,bmw,nein,29-03-2016 00:00,0.0,34582.0,05-04-2016 23:47
101233,08-03-2016 12:47,Ford_Focus_C_MAX_1.6_85kw_Bj_05_2.Hand_Klima_T...,privat,Angebot,2350.0,test,bus,2005.0,manuell,116.0,c_max,125000.0,0.0,benzin,ford,NaN,08-03-2016 00:00,0.0,13595.0,08-03-2016 12:47
101234,11-03-2016 10:43,BMW_116_i_Schwarz_Matt,privat,Angebot,4250.0,test,NaN,2017.0,manuell,115.0,1er,150000.0,10.0,benzin,bmw,NaN,11-03-2016 00:00,0.0,42477.0,11-03-2016 10:43


In [5]:
df=df.drop('seller',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [6]:
print(df.offerType.value_counts())

Angebot    101233
Gesuch          2
Name: offerType, dtype: int64


In [7]:
df[df.offerType !='Gesuch']

,dateCrawled,name,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,24-03-2016 11:52,Golf_3_1.6,Angebot,480.0,test,NaN,1993.0,manuell,0.0,golf,150000.0,0.0,benzin,volkswagen,NaN,24-03-2016 00:00,0.0,70435.0,07-04-2016 03:16
1,24-03-2016 10:58,A5_Sportback_2.7_Tdi,Angebot,18300.0,test,coupe,2011.0,manuell,190.0,NaN,125000.0,5.0,diesel,audi,ja,24-03-2016 00:00,0.0,66954.0,07-04-2016 01:46
2,14-03-2016 12:52,"Jeep_Grand_Cherokee_""Overland""",Angebot,9800.0,test,suv,2004.0,automatik,163.0,grand,125000.0,8.0,diesel,jeep,NaN,14-03-2016 00:00,0.0,90480.0,05-04-2016 12:47
3,17-03-2016 16:54,GOLF_4_1_4__3TÜRER,Angebot,1500.0,test,kleinwagen,2001.0,manuell,75.0,golf,150000.0,6.0,benzin,volkswagen,nein,17-03-2016 00:00,0.0,91074.0,17-03-2016 17:40
4,31-03-2016 17:25,Skoda_Fabia_1.4_TDI_PD_Classic,Angebot,3600.0,test,kleinwagen,2008.0,manuell,69.0,fabia,90000.0,7.0,diesel,skoda,nein,31-03-2016 00:00,0.0,60437.0,06-04-2016 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101231,20-03-2016 17:36,Opel_Mokka_1.4_Turbo_ecoFLEX_Start/Stop_Color_...,Angebot,20500.0,test,suv,2015.0,manuell,140.0,andere,10000.0,10.0,benzin,opel,nein,20-03-2016 00:00,0.0,65428.0,05-04-2016 19:47
101232,29-03-2016 14:42,BMW_325i,Angebot,4000.0,test,cabrio,1996.0,manuell,192.0,3er,150000.0,3.0,benzin,bmw,nein,29-03-2016 00:00,0.0,34582.0,05-04-2016 23:47
101233,08-03-2016 12:47,Ford_Focus_C_MAX_1.6_85kw_Bj_05_2.Hand_Klima_T...,Angebot,2350.0,test,bus,2005.0,manuell,116.0,c_max,125000.0,0.0,benzin,ford,NaN,08-03-2016 00:00,0.0,13595.0,08-03-2016 12:47
101234,11-03-2016 10:43,BMW_116_i_Schwarz_Matt,Angebot,4250.0,test,NaN,2017.0,manuell,115.0,1er,150000.0,10.0,benzin,bmw,NaN,11-03-2016 00:00,0.0,42477.0,11-03-2016 10:43


In [8]:
df=df.drop('offerType',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [9]:
print(df.shape)
df=df[(df.powerPS > 50) & (df.powerPS < 900)]
print(df.shape)

(101236, 18)
(86917, 18)


In [10]:
df=df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]
print(df.shape)

(84154, 18)


In [ ]:
df.drop(['name', 'abtest', 'dateCrawled', 'nrOfpictures', 'lastseen', 'postalcode', 'dateCreated'], axis='columns', inplace=True)

In [12]:
new_df = df.copy()
new_df = new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','model','kilometer','monthOfRegistration','fuelType','notRepairedDamage'])
print(new_df)

             dateCrawled                                               name  \
1       24-03-2016 10:58                               A5_Sportback_2.7_Tdi   
2       14-03-2016 12:52                     Jeep_Grand_Cherokee_"Overland"   
3       17-03-2016 16:54                                 GOLF_4_1_4__3TÜRER   
4       31-03-2016 17:25                     Skoda_Fabia_1.4_TDI_PD_Classic   
5       04-04-2016 17:36  BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...   
...                  ...                                                ...   
101229  05-03-2016 20:46                                 E30_318i_TÜV_09/16   
101230  19-03-2016 02:42                                           BMW_316i   
101231  20-03-2016 17:36  Opel_Mokka_1.4_Turbo_ecoFLEX_Start/Stop_Color_...   
101232  29-03-2016 14:42                                           BMW_325i   
101233  08-03-2016 12:47  Ford_Focus_C_MAX_1.6_85kw_Bj_05_2.Hand_Klima_T...   

          price   abtest vehicleType  yearOfRegistr

In [13]:
new_df.gearbox.replace(('manuell', 'automatik'), ('manual', 'automatic'), inplace=True)
new_df.fuelType.replace(('benzin', 'andere', 'elektro'), ('petrol', 'others', 'electric'), inplace=True)
new_df.vehicleType.replace(('kleinwagen', 'cabrio', 'kombi', 'andere'),('small car', 'convertible', 'combination', 'others'), inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [14]:
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]
print(new_df)

             dateCrawled                                               name  \
1       24-03-2016 10:58                               A5_Sportback_2.7_Tdi   
2       14-03-2016 12:52                     Jeep_Grand_Cherokee_"Overland"   
3       17-03-2016 16:54                                 GOLF_4_1_4__3TÜRER   
4       31-03-2016 17:25                     Skoda_Fabia_1.4_TDI_PD_Classic   
5       04-04-2016 17:36  BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...   
...                  ...                                                ...   
101229  05-03-2016 20:46                                 E30_318i_TÜV_09/16   
101230  19-03-2016 02:42                                           BMW_316i   
101231  20-03-2016 17:36  Opel_Mokka_1.4_Turbo_ecoFLEX_Start/Stop_Color_...   
101232  29-03-2016 14:42                                           BMW_325i   
101233  08-03-2016 12:47  Ford_Focus_C_MAX_1.6_85kw_Bj_05_2.Hand_Klima_T...   

          price   abtest  vehicleType  yearOfRegist

In [15]:
new_df['notRepairedDamage'].fillna (value='not-declared', inplace=True)
new_df['fuelType'].fillna (value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna (value='not-declared', inplace=True)
new_df['model'].fillna (value='not-declared', inplace=True)

In [16]:
new_df.to_csv("auto_preprocessed.csv")

In [17]:
labels=['gearbox','notRepairedDamage','model','brand','fuelType','vehicleType']

In [18]:
mapper={}
for i in labels:
  mapper[i]=LabelEncoder()
  mapper[i].fit(new_df[i])
  tr=mapper[i].transform(new_df[i])
  np.save(str('classes'+i+' .npy'), mapper[i].classes_)
  print(i,":",mapper[i])
  new_df.loc[:,i+ '_labels'] = pd.Series(tr, index=new_df.index)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()


In [19]:
labeled= new_df[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration'] + [x+"_labels" for x in labels]]
print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


**Spliting data into dependent and independent variables**

In [20]:
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values
Y = Y.reshape(-1,1)

In [21]:
Y

array([[18300.],
       [ 9800.],
       [ 1500.],
       ...,
       [20500.],
       [ 4000.],
       [ 2350.]])

In [22]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 3)


**Choose The Appropriate Model**

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=34)
print(regressor)

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)


In [24]:
regressor.fit(Y_train, np.ravel(Y_train,order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

**Check The Metrics Of The Model**

In [25]:
y_pred = regressor.predict(Y_test)
print(r2_score (Y_test,y_pred))

0.9999798703290252


**Save The Model**

In [26]:
filename = 'resale_model.sav' 
pickle.dump(regressor, open(filename, 'wb'))

In [87]:
!pip install ibm_watson_machine_learning

In [102]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url":"https://us-south.ml.cloud.ibm.com","instance_id": "openshift",'version':'4.0'
                    ,"apikey":"hL8g5FipFw5NQa5QJlDEnURDIR2IIP_NLn2M5EWfY98N"
                  }

client = APIClient(wml_credentials)

WMLClientError: `username` missing in wml_credentials.

In [103]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item ['entity']["name"]== space_name)['metadata']['id'])

In [104]:
space_uid=guid_from_space_name(client,'car')
print("Space UID="+space_uid)

Space UID=75ee7faf-7fb0-4511-8f3c-ee9f81a15c49


In [105]:
client.set.default_space(space_uid)

'SUCCESS'

In [14]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [52]:
software_spec_uid=client.software_specifications.get_uid_by_name("default_py3.7")
software_spec_uid

'e4429883-c883-42b6-87a8-f419d64088cd'

In [54]:
model_details=client.repository.store_model(model=regressor,meta_props={
client.repository.ModelMetaNames.NAME: "car_resale_value_prediction",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.23"})


Traceback (most recent call last):
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 254, in _save_scikit_pipeline_model
    model_artifact = self._create_pipeline_model(artifact, query_param)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 514, in _create_pipeline_model
    model_artifact = self._create_pipeline_model_v4_cloud(model_artifact, query_param)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/mlrepositoryclient/model_collection.py", line 1005, in _create_pipeline_model_v4_cloud
    model_output = self.repository_api.ml_assets_model_creation_v4_cloud(model_input, query_param,headers)
  File "/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/libs/repo/swagger_client/apis/repository_api.py", line 301,

WMLClientError: Publishing model failed.
Reason: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 19 Nov 2022 09:15:04 GMT', 'Content-Type': 'text/plain; charset=UTF-8', 'Content-Length': '274', 'Connection': 'keep-alive', 'server-timing': 'intid;desc=39589577de8597f4, intid;desc=39589577de8597f4', 'WWW-Authenticate': 'Bearer realm="IAM"', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'CF-Cache-Status': 'DYNAMIC', 'Server': 'cloudflare', 'CF-RAY': '76c7d6d8fe7b0b95-DFW'})
HTTP response body: {
  "trace": "076f3aa22124b540dbf92e86bb13f21a",
  "errors": [{
    "code": "authorization_rejected",
    "message": "The token is expired since 2022-11-19T09:12:51Z",
    "target": {
      "type": "header",
      "name": "Authorization"
    }
  }],
  "status_code": "401"
}


In [47]:
model_id=client.repository.get_model_uid(model_details)

NameError: name 'model_details' is not defined